In [3]:
import pandas as pd
from datetime import datetime, timedelta
import requests
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

In [4]:
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.options.display.float_format='{:,.2f}'.format

# ***Parte A***

### **Conexion con drive google**

In [ ]:
#from google.colab import drive

In [ ]:
#drive.mount('/content/drive', force_remount=True)

In [ ]:
#%cd '/content/drive/MyDrive/Colab Notebooks/data/06_open_pay'
#!ls

### **Contrucción de Dataframe Nasdaq sin excluded symbols**

In [3]:
df_nasdaq = pd.read_csv('NASDAQ.txt', names=['Symbol', 'Description'], skiprows=1, sep='\t')

In [4]:
df_nasdaq.shape

(3428, 2)

In [ ]:
#df_nasdaq['Symbol'].value_counts()

In [ ]:
#df_nasdaq.describe()

In [5]:
df_excluded_symbols = pd.read_csv('excluded_symbols.txt', names=['Symbol'])

In [6]:
df_excluded_symbols['Symbol'] = df_excluded_symbols['Symbol'].str.replace('[^\w\s]','')

C:\Users\LuisGómez\AppData\Local\Temp\ipykernel_11256\825134795.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_excluded_symbols['Symbol'] = df_excluded_symbols['Symbol'].str.replace('[^\w\s]','')


In [7]:
df_excluded_symbols.shape

(283, 1)

In [ ]:
#df_excluded_symbols['Symbol'].value_counts()

In [8]:
df_stock_unificadas = pd.merge(df_nasdaq, df_excluded_symbols, on=['Symbol'], how='inner')

In [9]:
df_stock_unificadas.shape

(18, 2)

In [ ]:
#df_stock_unificadas['Symbol'].value_counts()

In [10]:
df_stock_validado = df_nasdaq[~df_nasdaq['Symbol'].isin(df_stock_unificadas['Symbol'])]

In [11]:
df_stock_validado.shape

(3410, 2)

In [ ]:
#df_stock_validado.value_counts()

### **Construir Dataframe de la carpeta full_history**

In [ ]:
#%cd '/content/drive/MyDrive/Colab Notebooks/data/06_open_pay/00_full_history'
#!ls

In [65]:
#cd full_history

In [66]:
#pwd

In [14]:
res = []
for path in os.listdir('.'):
    if os.path.isfile(os.path.join(path)):
        res.append(path)

In [16]:
len(res)

8188

In [17]:
list_valores = set(df_stock_validado['Symbol'])

In [19]:
len(list_valores)

3410

In [20]:
files_revision = { symbol+".csv" for symbol in list_valores}

In [21]:
res = set(res)

In [22]:
len(res)

8188

In [23]:
dataframe_files = res.intersection(files_revision)

In [25]:
len(dataframe_files)

3410

In [26]:
file_globalpd = pd.DataFrame()
frames = []
for df_file in dataframe_files:
  file_one_by_one = pd.read_csv(df_file)
  symbol = df_file.split(".")[0]
  file_one_by_one['symbols'] = symbol
  frames.append(file_one_by_one)
file_globalpd= pd.concat(frames)


In [28]:
len(file_globalpd)

10182789

In [29]:
file_globalpd.head(5)

,date,volume,open,close,high,low,adjclose,symbols
0,2019-04-18,297500,1.96,1.97,1.97,1.91,1.97,TUES
1,2019-04-17,264500,1.94,1.94,2.00,1.91,1.94,TUES
2,2019-04-16,316600,1.91,1.93,1.97,1.91,1.93,TUES
3,2019-04-15,424900,1.92,1.91,1.98,1.91,1.91,TUES
4,2019-04-12,357100,2.02,1.92,2.02,1.92,1.92,TUES


In [30]:
file_globalpd.loc[(file_globalpd['symbols'].eq('ZSAN'))].tail(5)

,date,volume,open,close,high,low,adjclose,symbols
1060,2015-02-02,1700,224.20,221.80,225.96,218.60,221.80,ZSAN
1061,2015-01-30,6600,224.00,221.80,227.40,218.00,221.80,ZSAN
1062,2015-01-29,12300,221.60,224.00,227.40,219.80,224.00,ZSAN
1063,2015-01-28,7700,221.40,221.00,228.00,220.00,221.00,ZSAN
1064,2015-01-27,78200,243.20,220.20,248.00,219.80,220.20,ZSAN


In [44]:
file_globalpd['date'] = file_globalpd['date'].apply(pd.to_datetime)

In [45]:
file_globalpd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10182789 entries, 0 to 6973
Data columns (total 8 columns):
 #   Column    Dtype         
---  ------    -----         
 0   date      datetime64[ns]
 1   volume    object        
 2   open      float64       
 3   close     float64       
 4   high      float64       
 5   low       float64       
 6   adjclose  float64       
 7   symbols   object        
dtypes: datetime64[ns](1), float64(5), object(2)
memory usage: 699.2+ MB


In [49]:
file_globalpd = file_globalpd.query("date >= '2019-01-01'")

In [50]:
file_globalpd.head(1000)

,date,volume,open,close,high,low,adjclose,symbols
0,2019-04-18,297500,1.96,1.97,1.97,1.91,1.97,TUES
1,2019-04-17,264500,1.94,1.94,2.00,1.91,1.94,TUES
2,2019-04-16,316600,1.91,1.93,1.97,1.91,1.93,TUES
3,2019-04-15,424900,1.92,1.91,1.98,1.91,1.91,TUES
4,2019-12-04,357100,2.02,1.92,2.02,1.92,1.92,TUES
...,...,...,...,...,...,...,...,...
19,2019-03-22,119200,29.48,28.58,29.56,28.49,28.58,HCM
20,2019-03-21,117000,29.13,29.71,29.86,29.13,29.71,HCM
21,2019-03-20,170100,28.99,29.35,29.49,28.84,29.35,HCM
22,2019-03-19,98800,28.43,29.00,29.27,28.43,29.00,HCM


In [51]:
file_globalpd.shape

(239301, 8)

In [64]:
file_globalpd.to_csv(r'../output_files/df_full_history_2019.csv', index = False, header=True)

### **Leer API y Promedio tipo de cambio**

In [30]:
idSerie = "SF43718"

In [31]:
fechaFin = datetime.now()


In [32]:
fechaIni = fechaFin.replace(day=1)

In [33]:
fechaFin = fechaFin.strftime("%Y-%m-%d")
fechaIni = fechaIni.strftime("%Y-%m-%d")

In [34]:
url = f"https://www.banxico.org.mx/SieAPIRest/service/v1/series/{idSerie}/datos/{fechaIni}/{fechaFin}"


In [35]:
params = {
    "token" : "91eb8cb5c261f9b0a5b4dead1fb08296570996db268fcaa6e34c04b52552ace8",
}

In [36]:
response = requests.get(url, params=params)

In [37]:
tipo_de_cambio = response.json().get('bmx')

In [38]:
series = tipo_de_cambio.get('series')
#series

In [39]:
datos = series[0].get('datos')
#datos

In [40]:
usd_monthly_avg = np.average([float(dato.get('dato')) for dato in datos])
usd_monthly_avg

20.013668421052632

# ***Parte B***

### **Leer y corregir formato**

In [80]:
#cd ..

In [210]:
from utils.constants import READ_CSV_CONFIGS

In [211]:
dtypes = {'Mes Operacion': 'str'}
parse_dates = ['Mes Operacion']
#df_estado_transacciones = pd.read_csv("Examen Parte B.csv", dtype=dtypes, parse_dates=parse_dates)
df_estado_transacciones = pd.read_csv("Examen Parte B.csv")

In [212]:
df_estado_transacciones

,Comercio Id,Mes Operacion,Estado,Moneda,# Transacciones,Monto Transaccion,Monto Contracargo (Venta)
0,0,01/07/21,IN_PROGRESS,MXN,"3,679,780","11,928,910,467",0
1,0,01/07/21,COMPLETED,MXN,"12,366","23,472,915",0
2,0,01/07/21,FAILED,MXN,"5,115","7,124,189.72",0
3,0,01/08/21,IN_PROGRESS,MXN,"2,536,306","8,193,985,268",0
4,0,01/08/21,CANCELLED,MXN,"876,854","2,786,923,528",0
...,...,...,...,...,...,...,...
1995,259,01/08/21,FAILED,MXN,"1,636","1,966,402.39",0
1996,259,01/08/21,COMPLETED,MXN,826,"991,902.66","9,154"
1997,259,01/08/21,CANCELLED,MXN,1,"217,369.92",0
1998,260,01/07/21,COMPLETED,MXN,"1,713","4,374,585.07",0


In [213]:
df_estado_transacciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Comercio Id                2000 non-null   int64 
 1   Mes Operacion              2000 non-null   object
 2   Estado                     2000 non-null   object
 3   Moneda                     2000 non-null   object
 4   # Transacciones            2000 non-null   object
 5   Monto Transaccion          2000 non-null   object
 6   Monto Contracargo (Venta)  2000 non-null   object
dtypes: int64(1), object(6)
memory usage: 109.5+ KB


In [214]:
df_estado_transacciones.rename(columns = READ_CSV_CONFIGS.get("columns"), inplace = True)

In [215]:
#df_estado_transacciones['num_transacciones'] = df_estado_transacciones['num_transacciones'].str.replace(r'[^\d*\.?\d*]+', '', regex=True)
df_estado_transacciones['monto_transaccion'] = df_estado_transacciones['monto_transaccion'].str.replace(r'[^\d*\.?\d*]+', '', regex=True)
df_estado_transacciones['monto_contracargo_venta'] = df_estado_transacciones['monto_contracargo_venta'].str.replace(r'[^\d*\.?\d*]+', '', regex=True)

In [216]:
df_estado_transacciones

,comercio_id,mes_operacion,estado,moneda,num_transacciones,monto_transaccion,monto_contracargo_venta
0,0,01/07/21,IN_PROGRESS,MXN,"3,679,780",11928910467,0
1,0,01/07/21,COMPLETED,MXN,"12,366",23472915,0
2,0,01/07/21,FAILED,MXN,"5,115",7124189.72,0
3,0,01/08/21,IN_PROGRESS,MXN,"2,536,306",8193985268,0
4,0,01/08/21,CANCELLED,MXN,"876,854",2786923528,0
...,...,...,...,...,...,...,...
1995,259,01/08/21,FAILED,MXN,"1,636",1966402.39,0
1996,259,01/08/21,COMPLETED,MXN,826,991902.66,9154
1997,259,01/08/21,CANCELLED,MXN,1,217369.92,0
1998,260,01/07/21,COMPLETED,MXN,"1,713",4374585.07,0


In [217]:
df_estado_transacciones = df_estado_transacciones.astype({
        "comercio_id":"str", 
        "monto_transaccion":"float64", 
        "monto_contracargo_venta":"float64"
})

In [218]:
df_estado_transacciones

,comercio_id,mes_operacion,estado,moneda,num_transacciones,monto_transaccion,monto_contracargo_venta
0,0,01/07/21,IN_PROGRESS,MXN,"3,679,780","11,928,910,467.00",0.00
1,0,01/07/21,COMPLETED,MXN,"12,366","23,472,915.00",0.00
2,0,01/07/21,FAILED,MXN,"5,115","7,124,189.72",0.00
3,0,01/08/21,IN_PROGRESS,MXN,"2,536,306","8,193,985,268.00",0.00
4,0,01/08/21,CANCELLED,MXN,"876,854","2,786,923,528.00",0.00
...,...,...,...,...,...,...,...
1995,259,01/08/21,FAILED,MXN,"1,636","1,966,402.39",0.00
1996,259,01/08/21,COMPLETED,MXN,826,"991,902.66","9,154.00"
1997,259,01/08/21,CANCELLED,MXN,1,"217,369.92",0.00
1998,260,01/07/21,COMPLETED,MXN,"1,713","4,374,585.07",0.00


In [219]:
df_estado_transacciones['moneda'].value_counts()

MXN    1855
COP      52
USD      29
EUR      18
JPY      12
CLP      10
PKR       8
AED       8
TWD       6
ARS       2
Name: moneda, dtype: int64

In [220]:
df_estado_transacciones['estado'].value_counts()

COMPLETED         523
FAILED            523
ERROR             298
CANCELLED         272
IN_PROGRESS       223
EXPIRED           124
CHARGE_PENDING     37
Name: estado, dtype: int64

### **Filtrar solo las monedas correspondientes a MXN y USD**

In [221]:
df_transacciones_usd_mxn = df_estado_transacciones.loc[(df_estado_transacciones['moneda'].isin(['MXN', 'USD']))]

### **Transformación de Dolores a Pesos**

In [156]:
usd_monthly_avg.dtype

dtype('float64')

In [157]:
usd_monthly_avg

20.013668421052632

In [158]:
df_transacciones_usd_mxn['monto_transaccion'].loc[(df_transacciones_usd_mxn['moneda'].eq('USD'))] = df_transacciones_usd_mxn['monto_transaccion'].loc[(df_transacciones_usd_mxn['moneda'].eq('USD'))] * usd_monthly_avg
df_transacciones_usd_mxn['monto_contracargo_venta'].loc[(df_transacciones_usd_mxn['moneda'].eq('USD'))] = df_transacciones_usd_mxn['monto_contracargo_venta'].loc[(df_transacciones_usd_mxn['moneda'].eq('USD'))] * usd_monthly_avg

In [159]:
df_transacciones_usd_mxn.loc[(df_transacciones_usd_mxn['moneda'].eq('USD'))].head(5)

,comercio_id,mes_operacion,estado,moneda,num_transacciones,monto_transaccion,monto_contracargo_venta
524,65,01/07/21,COMPLETED,USD,299,"2,716,270.09","33,435.64"
526,65,01/07/21,FAILED,USD,143,"868,517.76",0.00
529,65,01/07/21,IN_PROGRESS,USD,4,"2,873.76",0.00
531,65,01/07/21,CANCELLED,USD,4,"16,307.34",0.00
533,65,01/07/21,ERROR,USD,4,"9,031.37",0.00


### **Monto total de transacciones completadas y falladas**

In [324]:
#pwd

In [336]:
#df_transacciones_usd_mxn.to_csv(r'output_files/df_usd_mxn.csv', header=True)

In [163]:
#df_transacciones_usd_mxn

In [164]:
totales_monto_transaccion = df_transacciones_usd_mxn.pivot_table(index=['estado'], values=['monto_transaccion'], aggfunc='sum')

In [165]:
totales_monto_transaccion

,monto_transaccion
estado,
CANCELLED,"3,249,901,275.90"
CHARGE_PENDING,"17,627,547.23"
COMPLETED,"12,989,564,391.66"
ERROR,"61,989,902.31"
EXPIRED,"498,671,466.85"
FAILED,"9,193,061,589.99"
IN_PROGRESS,"30,391,913,181.03"


In [222]:
total_monto_transaccion_completed = df_transacciones_usd_mxn['monto_transaccion'].loc[(df_transacciones_usd_mxn['estado'].eq('COMPLETED'))].sum()

In [223]:
print(f'El monto total de las transacciones completadas es {total_monto_transaccion_completed:,.2f}')

El monto total de las transacciones completadas es 12,948,060,743.42


In [224]:
total_monto_transaccion_failed = df_transacciones_usd_mxn['monto_transaccion'].loc[(df_transacciones_usd_mxn['estado'].eq('FAILED'))].sum()

In [225]:
print(f'El monto total de las transacciones completadas es {total_monto_transaccion_failed:,.2f}')

El monto total de las transacciones completadas es 9,171,939,283.50


### **Existe algún comercio que nunca haya completado transacciones**

In [226]:
transacciones_completed = set(df_transacciones_usd_mxn['comercio_id'].loc[df_transacciones_usd_mxn['estado'].isin(['COMPLETED'])])

In [171]:
len(transacciones_completed)

248

In [172]:
total_comercios = set(df_transacciones_usd_mxn['comercio_id'])

In [173]:
len(total_comercios)

251

In [174]:
comercios_sin_estado_completado = transacciones_completed.symmetric_difference(total_comercios)

In [175]:
len(comercios_sin_estado_completado)

3

In [176]:
comercios_sin_estado_completado

{'100', '113', '134'}

In [177]:
df_comercios_sin_estado_completado = df_transacciones_usd_mxn.loc[(df_transacciones_usd_mxn['comercio_id'].isin(['100','113','134']))]

In [178]:
df_comercios_sin_estado_completado

,comercio_id,mes_operacion,estado,moneda,num_transacciones,monto_transaccion,monto_contracargo_venta
804,100,01/07/21,FAILED,MXN,"6,800","5,123,429.03",0.00
805,100,01/07/21,CANCELLED,MXN,"5,535","6,857,684.84","540,740.00"
806,100,01/07/21,CANCELLED,MXN,5,"5,450.00",0.00
807,100,01/07/21,ERROR,MXN,5,"5,050.00",0.00
808,100,01/08/21,FAILED,MXN,"5,137","4,145,929.82",0.00
809,100,01/08/21,ERROR,MXN,"4,592","6,007,893.10","12,183.00"
810,100,01/08/21,ERROR,MXN,9,"16,140.00",0.00
913,113,01/07/21,FAILED,MXN,"5,512","12,889,785.85","213,067.24"
914,113,01/07/21,FAILED,MXN,"3,497","12,841,244.64",0.00
915,113,01/07/21,ERROR,MXN,6,"35,694.37",0.00


### **Calcule el porcentaje de transacciones completadas en cada uno de los comercios**

In [307]:
porc_transacciones_completed = df_transacciones_usd_mxn.loc[df_transacciones_usd_mxn['estado'].isin(['COMPLETED'])]

In [308]:
porc_transacciones_completed

,comercio_id,mes_operacion,estado,moneda,num_transacciones,monto_transaccion,monto_contracargo_venta
1,0,01/07/21,COMPLETED,MXN,"12,366","23,472,915.00",0.00
5,0,01/08/21,COMPLETED,MXN,"11,693","22,654,486.00",0.00
8,1,01/07/21,COMPLETED,MXN,"8,218","9,188,680.00",0.00
11,1,01/08/21,COMPLETED,MXN,"8,433","10,123,512.00",0.00
14,2,01/07/21,COMPLETED,MXN,"190,740","14,991,421.40","292,951.89"
...,...,...,...,...,...,...,...
1987,258,01/07/21,COMPLETED,MXN,796,"142,870.00",0.00
1989,258,01/08/21,COMPLETED,MXN,924,"162,689.00",0.00
1993,259,01/07/21,COMPLETED,MXN,416,"552,886.86","8,389.00"
1996,259,01/08/21,COMPLETED,MXN,826,"991,902.66","9,154.00"


In [275]:
Total_transacciones_completadas = porc_transacciones_completed.shape[0]

In [276]:
Total_transacciones_completadas

496

In [277]:
porc_transacciones_completed['Total_transacciones_completadas'] = Total_transacciones_completadas

In [278]:
porc_transacciones_completed  

,comercio_id,mes_operacion,estado,moneda,num_transacciones,monto_transaccion,monto_contracargo_venta,Total_transacciones_completadas
1,0,01/07/21,COMPLETED,MXN,"12,366","23,472,915.00",0.00,496
5,0,01/08/21,COMPLETED,MXN,"11,693","22,654,486.00",0.00,496
8,1,01/07/21,COMPLETED,MXN,"8,218","9,188,680.00",0.00,496
11,1,01/08/21,COMPLETED,MXN,"8,433","10,123,512.00",0.00,496
14,2,01/07/21,COMPLETED,MXN,"190,740","14,991,421.40","292,951.89",496
...,...,...,...,...,...,...,...,...
1987,258,01/07/21,COMPLETED,MXN,796,"142,870.00",0.00,496
1989,258,01/08/21,COMPLETED,MXN,924,"162,689.00",0.00,496
1993,259,01/07/21,COMPLETED,MXN,416,"552,886.86","8,389.00",496
1996,259,01/08/21,COMPLETED,MXN,826,"991,902.66","9,154.00",496


In [279]:
porcentaje_por_comercio = (porc_transacciones_completed.groupby(['comercio_id']).apply(lambda x: x['comercio_id'].count()
                                    / Total_transacciones_completadas) * 100).to_dict()

In [137]:
porcentaje_por_comercio

{'0': 0.4032258064516129,
 '1': 0.4032258064516129,
 '10': 0.4032258064516129,
 '101': 0.4032258064516129,
 '102': 0.4032258064516129,
 '103': 0.4032258064516129,
 '104': 0.4032258064516129,
 '105': 0.4032258064516129,
 '106': 0.4032258064516129,
 '107': 0.4032258064516129,
 '108': 0.4032258064516129,
 '109': 0.4032258064516129,
 '11': 0.4032258064516129,
 '110': 0.4032258064516129,
 '111': 0.4032258064516129,
 '112': 0.4032258064516129,
 '115': 0.4032258064516129,
 '116': 0.4032258064516129,
 '117': 0.4032258064516129,
 '118': 0.4032258064516129,
 '119': 0.4032258064516129,
 '12': 0.4032258064516129,
 '120': 0.4032258064516129,
 '121': 0.4032258064516129,
 '122': 0.4032258064516129,
 '123': 0.4032258064516129,
 '124': 0.4032258064516129,
 '125': 0.4032258064516129,
 '126': 0.4032258064516129,
 '127': 0.4032258064516129,
 '128': 0.4032258064516129,
 '129': 0.4032258064516129,
 '130': 0.4032258064516129,
 '131': 0.4032258064516129,
 '132': 0.4032258064516129,
 '133': 0.4032258064516129,

In [309]:
porc_transacciones_completed['porcentaje_por_comercio'] = porc_transacciones_completed['comercio_id'].map(porcentaje_por_comercio)
porc_transacciones_completed

,comercio_id,mes_operacion,estado,moneda,num_transacciones,monto_transaccion,monto_contracargo_venta,porcentaje_por_comercio
1,0,01/07/21,COMPLETED,MXN,"12,366","23,472,915.00",0.00,0.40
5,0,01/08/21,COMPLETED,MXN,"11,693","22,654,486.00",0.00,0.40
8,1,01/07/21,COMPLETED,MXN,"8,218","9,188,680.00",0.00,0.40
11,1,01/08/21,COMPLETED,MXN,"8,433","10,123,512.00",0.00,0.40
14,2,01/07/21,COMPLETED,MXN,"190,740","14,991,421.40","292,951.89",0.40
...,...,...,...,...,...,...,...,...
1987,258,01/07/21,COMPLETED,MXN,796,"142,870.00",0.00,0.40
1989,258,01/08/21,COMPLETED,MXN,924,"162,689.00",0.00,0.40
1993,259,01/07/21,COMPLETED,MXN,416,"552,886.86","8,389.00",0.40
1996,259,01/08/21,COMPLETED,MXN,826,"991,902.66","9,154.00",0.40


### **Calcule el porcentaje de transacciones completadas mensuales de cada uno de los comercios y posterior a ello calcule el promedio.**

In [310]:
porc_transacciones_completed['mes_operacion'].value_counts()

01/07/21    249
01/08/21    247
Name: mes_operacion, dtype: int64

In [329]:
porc_transacciones_completed['mes_operacion'] = pd.to_datetime(porc_transacciones_completed['mes_operacion']).dt.normalize()

In [331]:
porc_transacciones_completed['mes'] = porc_transacciones_completed['mes_operacion'].dt.day

In [424]:
porc_transacciones_completed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 496 entries, 1 to 1998
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   comercio_id              496 non-null    object        
 1   mes_operacion            496 non-null    datetime64[ns]
 2   estado                   496 non-null    object        
 3   moneda                   496 non-null    object        
 4   num_transacciones        496 non-null    object        
 5   monto_transaccion        496 non-null    float64       
 6   monto_contracargo_venta  496 non-null    float64       
 7   porcentaje_por_comercio  496 non-null    float64       
 8   mes                      496 non-null    int64         
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 38.8+ KB


In [333]:
porc_transacciones_completed['mes'].value_counts()

7    249
8    247
Name: mes, dtype: int64

In [418]:
porc_transacciones_completed.pivot_table('estado', index='comercio_id', columns='mes', aggfunc=[ lambda x: len(x) / Total_transacciones_completadas*100])

<lambda>     
mes                7    8
comercio_id              
0               0.20 0.20
1               0.20 0.20
10              0.20 0.20
101             0.20 0.20
102             0.20 0.20
...              ...  ...
95              0.20 0.20
96              0.20 0.20
97              0.20 0.20
98              0.20 0.20
99              0.20 0.20

[248 rows x 2 columns]

In [423]:
porc_transacciones_completed.pivot_table('estado',columns='mes', aggfunc=[ lambda x: len(x) / Total_transacciones_completadas*100])

<lambda>      
mes           7     8
estado    50.20 49.80

### **variables dummy**

In [441]:
pd.set_option('display.max_rows', None)

In [435]:
df_transacciones_usd_mxn['mes_operacion'] = pd.to_datetime(df_transacciones_usd_mxn['mes_operacion']).dt.normalize()

In [438]:
df_transacciones_usd_mxn['mes'] = df_transacciones_usd_mxn['mes_operacion'].dt.day

In [450]:
agg_func_count = {'estado': ['nunique']}
v_dummy = pd.pivot_table(df_transacciones_usd_mxn,
            index=['comercio_id'],
            columns=['mes','estado'],
            aggfunc=agg_func_count,
                        fill_value=0)

In [451]:
v_dummy

estado                                                \
              nunique                                                 
mes                 7                                                 
estado      CANCELLED CHARGE_PENDING COMPLETED ERROR EXPIRED FAILED   
comercio_id                                                           
0                   0              0         1     0       0      1   
1                   0              0         1     0       0      1   
10                  0              0         1     1       1      1   
100                 1              0         0     1       0      1   
101                 1              0         1     0       0      1   
102                 0              0         1     1       1      1   
103                 1              1         1     1       1      1   
104                 0              0         1     1       0      1   
105                 1              0         1     1       0      1   
106                 1              0         1     0       0      1   
107                 1              0         1     0       0      1   
108                 1              0         1     1       0      1   
109                 0              0         1     0       0      1   
11                  1              1         1     1       1      1   
110                 1              0         1     1       1      1   
111                 0              0         1     1       0      1   
112                 0              1         1     1       1      1   
113                 1              0         0     1       0      1   
115                 1              0         1     1       0      1   
116                 1              0         1     1       0      1   
117                 0              0         1     1       1      1   
118                 0              0         1     0       0      1   
119                 0              0         1     1       0      1   
12                  0              0         1     1       0      1   
120                 0              0         1     1       0      1   
121                 1              0         1     1       0      1   
122                 1              0         1     1       0      1   
123                 0              0         1     1       0      1   
124                 1              0         1     1       0      1   
125                 1              0         1     1       0      1   
126                 0              0         1     0       0      1   
127                 0              0         1     1       0      1   
128                 0              0         1     0       0      1   
129                 1              0         1     1       0      1   
130                 0              0         1     0       0      1   
131                 1              0         1     0       0      1   
132                 0              0         1     1       1      1   
133                 1              0         1     0       0      1   
134                 1              0         0     1       0      1   
135                 1              0         1     1       0      1   
136                 1              0         1     1       1      1   
137                 1              0         1     0       0      1   
138                 1              0         1     0       0      1   
139                 0              0         1     1       0      1   
14                  1              0         1     1       1      1   
140                 1              0         1     0       0      1   
141                 1              0         1     1       0      1   
142                 0              0         1     1       0      1   
143                 0              0         1     1       0      1   
144                 1              0         1     1       1      1   
146                 0              0         1     0       0      1   
147                 1              0   